# PROBLEMA 2: **Clasificación de Spam con Naive Bayes**


### Daniel Rojo Mata

## **Enunciado del problema**

Descarga el conjunto de datos de spam disponible en [este enlace](http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv) y realiza lo siguiente:

1. **Exploración de datos:**  
   - Reporta el porcentaje de correos etiquetados como *spam* y *no spam* en el conjunto de datos.

2. **División del conjunto de datos:**  
   - Divide aleatoriamente el conjunto de datos en:
     - **60 %** para entrenamiento  
     - **20 %** para validación  
     - **20 %** para prueba  
   - Usa `0` como semilla para el generador de números aleatorios.

3. **Entrenamiento de clasificadores:**  
   - Entrena **dos clasificadores Naive Bayes** con diferentes distribuciones.

4. **Evaluación de modelos:**  
   - Usa los clasificadores entrenados para predecir *spam* en los datos de **entrenamiento** y **validación**.  
   - Reporta el **porcentaje de predicciones correctas** de cada clasificador.

5. **Análisis del desempeño:**  
   - Discute el rendimiento de los diferentes clasificadores con base en los resultados obtenidos.

6. **Evaluación en el conjunto de prueba:**  
   - Reporta el **porcentaje de predicciones correctas** en el subconjunto de **prueba** utilizando el clasificador con mejor rendimiento en la validación.

## **Descripción del conjunto de datos**
El archivo `spam.csv` contiene **2001 valores por renglón**, distribuidos de la siguiente manera:
- Los **primeros 2000 valores** representan el **histograma de palabras** en un correo.
- El **último valor (posición 2000)** indica la clase del correo:
  - `1` → Es **spam**  
  - `0` → No es **spam**  


# **Solución**:

### Librerías

In [157]:
#Se importan los cosos necesarios 
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

### Clasificador con distribución Bernoulli

In [160]:
class NaiveBayesBernoulli:
    def __init__(self):
        """
        Inicializa los atributos del modelo de Naive Bayes con distribución Bernoulli.
        """
        self.num_clases = None  # Número de clases en el conjunto de datos
        self.num_caracteristicas = None  # Número de características en los datos
        self.proba_caracteristica_dado_clase = None  # Matriz de probabilidades condicionales P(característica|clase)
        self.proba_clase = None  # Vector de probabilidades a priori P(clase)

    def fit(self, X_ent, y_ent):
        """
        Entrena el modelo con el conjunto de entrenamiento.

        Parámetros:
            X_ent: Matriz de características del conjunto de entrenamiento
            y_ent: Vector de etiquetas de clase del conjunto de entrenamiento
        """
        clases_unicas = np.unique(y_ent)  # Encuentra las clases únicas en el conjunto de entrenamiento
        self.num_clases = clases_unicas.size  # Almacena el número total de clases
        self.num_caracteristicas = X_ent.shape[-1]  # Obtiene el número de características por muestra
        num_muestras = X_ent.shape[0]  # Obtiene el número total de muestras en el conjunto de entrenamiento

        # Inicializa la matriz de probabilidades condicionales con ceros
        self.proba_caracteristica_dado_clase = np.zeros((self.num_clases, self.num_caracteristicas))  
        # Inicializa el vector de probabilidades a priori con ceros
        self.proba_clase = np.zeros(self.num_clases)  

        for i, clase in enumerate(clases_unicas):  # Itera sobre cada clase
            X_clase = X_ent[y_ent == clase]  # Filtra las muestras pertenecientes a la clase actual
            cuenta_atributos = np.count_nonzero(X_clase, axis=0)  # Cuenta cuántas veces cada atributo es 1 dentro de la clase
                                                                  # axis = 0 indica que se aplica a lo largo de las columnas
            num_muestras_clase = X_clase.shape[0]  # Obtiene el número de muestras en la clase actual
            
            # Calcula P(atributo=1 | clase) dividiendo la cuenta entre el número de muestras de la clase
            self.proba_caracteristica_dado_clase[i, :] = cuenta_atributos / num_muestras_clase # [i, :] := modifica toda la fila 'i' de la matriz 
            
            # Calcula P(clase) como la proporción de muestras de la clase en el total de datos
            self.proba_clase[i] = num_muestras_clase / num_muestras  
    
    def calcular_probabilidades_posteriores(self, X):
        """
        Calcula las probabilidades posteriores (proporcionales) para cada muestra:
            P(clase|datos) ∝ P(datos|clase) * P(clase)
        Se omite el denominador P(datos) ya que es constante para todas las clases.

        Parámetros: 
            X: Matriz de características del conjunto de prueba
        
        return: 
            Matriz de probabilidades de cada clase para cada muestra
        """
        probabilidades = np.zeros((X.shape[0], self.num_clases))  # Matriz para almacenar las probabilidades calculadas
                                                                  # Es de tamaño: NumCaracterísticas x NumClases
        
        for indice_clase in range(self.num_clases):  # Itera sobre cada clase
            # Calcula la verosimilitud P(X|C) usando la distribución de Bernoulli.
                # Bajo el supuesto de independencia condicional (Bayes ingenuo), se calcula como:
                # P(X|C) = P(x_1|C) * P(x_2|C) * P(x_3|C) * ... * P(x_n|C)
            verosimilitud = np.prod(self.bernoulli(X, self.proba_caracteristica_dado_clase[indice_clase, :]), axis=1)  
            # Multiplica la verosimilitud por la probabilidad a priori de la clase P(C)
            # Esto nos da una cantidad proporcional a la probabilidad posterior P(C|X)
            probabilidades[:, indice_clase] = verosimilitud * self.proba_clase[indice_clase]

        return probabilidades  # Retorna la matriz de probabilidades posteriores

    def predict(self, X):
        """
        Predice las clases de un conjunto de datos.
        
        Parámetros:
            X: Matriz de características del conjunto de prueba
        
        return: 
            Vector con las clases predichas
        """
        return np.argmax(self.calcular_probabilidades_posteriores(X), axis=1)  # Retorna la clase con la mayor probabilidad posterior
    
    @staticmethod
    def bernoulli(x, q):
        """
        Función de masa de probabilidad de la distribución de Bernoulli.
        
        Parámetros:
            x: Valores de la variable aleatoria (0 o 1)
            q: Probabilidad del éxito (atributo = 1) en la distribución de Bernoulli
        
        return: 
            Probabilidad de cada muestra bajo la distribución de Bernoulli
        """
        epsilon = 1e-9  # Pequeño valor para evitar cálculos con 0 o 1 exactos que puedan generar errores numéricos
        q = np.clip(q, epsilon, 1 - epsilon)  # Restringe los valores de q al intervalo (epsilon, 1 - epsilon)
        return q**x * (1.0 - q)**(1.0 - x)  # Aplica la fórmula de la distribución de Bernoulli

### Clasificador con distribución Gaussiana

Se utiliza el logaritmo para un mejor trabajo, pues multiplicar probabilidades pequeñas puede dar valores muy bajos, cercanos a cero, lo que puede causar subdesbordamiento numérico en la computadora.

El objetivo de usar logaritmo es evitar esto último.

In [162]:
class NaiveBayesGaussiano:
    def __init__(self):
        """
        Inicializa los atributos del modelo de Naive Bayes con distribución Gaussiana.
        """
        self.media = None  # Media de cada característica por clase
        self.desviacion = None  # Desviación estándar de cada característica por clase
        self.proba_clase = None  # Probabilidad a priori de cada clase
        self.clases = None  # Clases únicas en el conjunto de datos
        self.epsilon = 1e-6  # Pequeño valor para evitar divisiones por 0 en la varianza

    def fit(self, X_ent, y_ent):
        """
        Entrena el modelo con el conjunto de entrenamiento.
        Calcula los parámetros de la distribución Gaussiana (media y desviación estándar) 
        para cada característica en cada clase.
        
        Parámetros:
            X_ent: ndarray de forma (num_muestras, num_características), matriz de datos de entrada.
            y_ent: ndarray de forma (num_muestras,), vector de etiquetas de clase.
        """
        self.clases = np.unique(y_ent)  # Obtiene las clases únicas en el conjunto de entrenamiento
        num_clases = len(self.clases)  # Número total de clases
        num_caracteristicas = X_ent.shape[1]  # Número de características en los datos
        
        # Inicializa matrices para almacenar la media, desviación estándar y probabilidades a priori, resp.
        self.media = np.zeros((num_clases, num_caracteristicas))
        self.desviacion = np.zeros((num_clases, num_caracteristicas))
        self.proba_clase = np.zeros(num_clases)
        
        # Itera sobre cada clase para calcular sus parámetros estadísticos
        for i, clase in enumerate(self.clases):
            X_clase = X_ent[y_ent == clase]  # Filtra las muestras que pertenecen a la clase actual
            self.media[i, :] = X_clase.mean(axis=0)  # Calcula la media de cada característica
            self.desviacion[i, :] = np.maximum(X_clase.std(axis=0), self.epsilon)  # Evita sigma = 0
            self.proba_clase[i] = len(X_clase) / len(X_ent)  # Calcula la probabilidad a priori de la clase

    def gaussian_log(self, x, mu, sigma):
        """
        Calcula la densidad de probabilidad de una distribución Gaussiana para cada característica.

        Parámetros:
            x: ndarray de forma (num_muestras, num_características), valores de entrada.
            mu: ndarray de forma (num_características,), media de la distribución Gaussiana.
            sigma: ndarray de forma (num_características,), desviación estándar de la distribución Gaussiana.

        return:
            Log-probabilidad de cada característica bajo la distribución Gaussiana correspondiente.
        """
        sigma = np.maximum(sigma, self.epsilon)  # Asegura que sigma no sea 0 para evitar errores numéricos
        coef = -0.5 * np.log(2 * np.pi * sigma**2)  # Cálculo del coeficiente de normalización en logaritmo
        exponente = -((x - mu) ** 2) / (2 * sigma ** 2)  # Cálculo de la parte exponencial de la función Gaussiana
        return coef + exponente  # Retorna la suma de log-probabilidades para estabilidad numérica

    def calcular_probabilidades_posteriores(self, X):
        """
        Calcula las probabilidades logarítmicas de cada clase para cada muestra.

        Parámetros:
            X: ndarray de forma (num_muestras, num_características), datos de entrada.

        return:
            log_probabilidades: ndarray de forma (num_muestras, num_clases),
                                matriz de log-probabilidades de cada muestra perteneciendo a cada clase.
        """
        num_muestras = X.shape[0]  # Número total de muestras
        num_clases = len(self.clases)  # Número total de clases
        log_probabilidades = np.zeros((num_muestras, num_clases))  # Inicializa matriz de log-probabilidades
        
        # Itera sobre cada clase para calcular la log-probabilidad de que una muestra pertenezca a ella
        for i in range(num_clases):
            log_prob_verosimilitud = self.gaussian_log(X, self.media[i, :], self.desviacion[i, :])  
            log_prob_verosimilitud = np.sum(log_prob_verosimilitud, axis=1)  # Suma las log-probabilidades de cada característica
            
            # Aplica la fórmula de la probabilidad posterior en logaritmo:
            # log P(C|X) = log P(X|C) + log P(C), sin dividir por P(X) ya que no afecta la clasificación
            log_probabilidades[:, i] = log_prob_verosimilitud + np.log(self.proba_clase[i]) 
        
        return log_probabilidades  # Retorna la matriz de log-probabilidades para cada muestra y clase

    def predict(self, X):
        """
        Predice la clase de cada muestra en X.

        Parámetros:
            X: ndarray de forma (num_muestras, num_características), datos de entrada.

        return:
            Un vector con las clases predichas para cada muestra.
        """
        log_probabilidades = self.calcular_probabilidades_posteriores(X)  # Calcula log-probabilidades para cada clase
        return self.clases[np.argmax(log_probabilidades, axis=1)]  # Retorna la clase con mayor log-probabilidad para cada muestra

### Procesamiento de la data

In [163]:
archivo = "spam.csv"
data = pd.read_csv(archivo, 
                   header=None,
                   sep=None,
                   engine="python")

In [164]:
# Separa las características y la etiqueta
X = data.iloc[:, :-1].to_numpy()  # Todas las columnas excepto la última
y = data.iloc[:, -1].to_numpy()   # La última columna (etiqueta)

### División del conjunto de datos

In [172]:
# Dividir en conjunto de entrenamiento (60%) y el resto (40%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)

# Dividir el conjunto temporal en validación (20%) y prueba (20%)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Ver las dimensiones de los conjuntos anteriores
print(f'Tamaño del conjunto de entrenamiento: {X_ent.shape}')
print(f'Tamaño del conjunto de validación: {X_val.shape}')
print(f'Tamaño del conjunto de prueba: {X_test.shape}')

Tamaño del conjunto de entrenamiento: (3103, 2000)
Tamaño del conjunto de validación: (1034, 2000)
Tamaño del conjunto de prueba: (1035, 2000)


## Resultados con Clasificador Bernoulli

In [166]:
# Crear una instancia de NaiveBayesBernoulli
nb_bernoulli = NaiveBayesBernoulli()

# Entrenar el clasificador con los datos de entrenamiento
nb_bernoulli.fit(X_ent, y_ent)

# Predecir para los conjuntos de datos
predicciones_ent = nb_bernoulli.predict(X_ent)
predicciones_val = nb_bernoulli.predict(X_val)
predicciones_test = nb_bernoulli.predict(X_test)

# Calcular la precisión (accuracy)
accuracy_ent_bernoulli = accuracy_score(y_ent, predicciones_ent) * 100
accuracy_val_bernoulli = accuracy_score(y_val, predicciones_val) * 100
accuracy_test_bernoulli = accuracy_score(y_test, predicciones_test) * 100

# Imprimir resultados
print(f"Precisión en entrenamiento: {accuracy_ent_bernoulli:.2f}%")
print(f"Precisión en validación: {accuracy_val_bernoulli:.2f}%")
print(f"Precisión en prueba: {accuracy_test_bernoulli:.2f}%")

Precisión en entrenamiento: 92.78%
Precisión en validación: 91.01%
Precisión en prueba: 91.50%


## Resultados con Clasificador Gaussiano

In [167]:
# Crear una instancia de NaiveBayesGaussiano
nb_gauss = NaiveBayesGaussiano()

# Entrenar el clasificador con los datos de entrenamiento
nb_gauss.fit(X_ent, y_ent)

# Predecir para los conjuntos de datos
predicciones_ent = nb_gauss.predict(X_ent)
predicciones_val = nb_gauss.predict(X_val)
predicciones_test = nb_gauss.predict(X_test)

# Calcular la precisión (accuracy)
accuracy_ent_gaussiano = accuracy_score(y_ent, predicciones_ent) * 100
accuracy_val_gaussiano = accuracy_score(y_val, predicciones_val) * 100
accuracy_test_gaussiano = accuracy_score(y_test, predicciones_test) * 100

# Imprimir resultados
print(f"Precisión en entrenamiento: {accuracy_ent:.2f}%")
print(f"Precisión en validación: {accuracy_val:.2f}%")
print(f"Precisión en prueba: {accuracy_test:.2f}%")

Precisión en entrenamiento: 93.81%
Precisión en validación: 93.04%
Precisión en prueba: 92.46%


## Resultados Con SckitLearn y Bernoulli

In [168]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score

# Dividir los datos en entrenamiento (60%), validación (20%) y prueba (20%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Crear una instancia del clasificador Naive Bayes Bernoulli
nb_classifier = BernoulliNB()

# Entrenar el modelo con los datos de entrenamiento
nb_classifier.fit(X_ent, y_ent)

# Predecir para los conjuntos de entrenamiento, validación y prueba
predicciones_ent = nb_classifier.predict(X_ent)
predicciones_val = nb_classifier.predict(X_val)
predicciones_test = nb_classifier.predict(X_test)

# Calcular la precisión (accuracy)
accuracy_ent = accuracy_score(y_ent, predicciones_ent) * 100
accuracy_val = accuracy_score(y_val, predicciones_val) * 100
accuracy_test = accuracy_score(y_test, predicciones_test) * 100

# Imprimir los resultados
print(f"Precisión en entrenamiento: {accuracy_ent:.2f}%")
print(f"Precisión en validación: {accuracy_val:.2f}%")
print(f"Precisión en prueba: {accuracy_test:.2f}%")


Precisión en entrenamiento: 91.11%
Precisión en validación: 91.10%
Precisión en prueba: 90.63%


## Resultados con ScikitLearn y Gaussiano

In [169]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# Dividir los datos en entrenamiento (60%), validación (20%) y prueba (20%)
X_ent, X_temp, y_ent, y_temp = train_test_split(X, y, test_size=0.4, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

# Crear una instancia del clasificador Naive Bayes Gaussiano
nb_classifier = GaussianNB()

# Entrenar el modelo con los datos de entrenamiento
nb_classifier.fit(X_ent, y_ent)

# Predecir para los conjuntos de entrenamiento, validación y prueba
predicciones_ent = nb_classifier.predict(X_ent)
predicciones_val = nb_classifier.predict(X_val)
predicciones_test = nb_classifier.predict(X_test)

# Calcular la precisión (accuracy)
accuracy_ent = accuracy_score(y_ent, predicciones_ent) * 100
accuracy_val = accuracy_score(y_val, predicciones_val) * 100
accuracy_test = accuracy_score(y_test, predicciones_test) * 100

# Imprimir los resultados
print(f"Precisión en entrenamiento: {accuracy_ent:.2f}%")
print(f"Precisión en validación: {accuracy_val:.2f}%")
print(f"Precisión en prueba: {accuracy_test:.2f}%")


Precisión en entrenamiento: 93.84%
Precisión en validación: 93.04%
Precisión en prueba: 92.46%


## Reporte de Spam y No Spam

In [170]:
spam_ratio = np.mean(y) * 100  # Asumiendo que spam = 1 y no spam = 0
print(f"Porcentaje de correos spam: {spam_ratio:.2f}%")
print(f"Porcentaje de correos no spam: {100 - spam_ratio:.2f}%")

Porcentaje de correos spam: 29.00%
Porcentaje de correos no spam: 71.00%


## Mejor Clasificador

In [171]:
# Se hace el comparativo con ambos clasificadores
# Se compara el "accuracy"

if accuracy_val_gaussiano > accuracy_val_bernoulli:
    mejor_clasificador = "Naive Bayes Gaussiano"
    mejor_modelo = nb_gauss
else:
    mejor_clasificador = "Naive Bayes Bernoulli"
    mejor_modelo = nb_bernoulli
print(f"El mejor clasificador en validación es: {mejor_clasificador}")

# Imprime el mejor resultado (no considerando los realizados por ScikitLearn) 
predicciones_test_mejor = mejor_modelo.predict(X_test)
accuracy_test_mejor = accuracy_score(y_test, predicciones_test_mejor) * 100
print(f"Precisión en prueba con {mejor_clasificador}: {accuracy_test_mejor:.2f}%")

El mejor clasificador en validación es: Naive Bayes Gaussiano
Precisión en prueba con Naive Bayes Gaussiano: 92.46%


## Discusión